# How to define a scikit-learn pipeline and visualize it

The goal of keeping this notebook is to:

- make it available for users that want to reproduce it locally
- archive the script in the event we want to rerecord this video with an
  update in the UI of scikit-learn in a future release.

### First we load the dataset

We need to define our data and target. In this case we will build a classification model

In [ ]:
import pandas as pd

ames_housing = pd.read_csv("../datasets/house_prices.csv", na_values='?')

target_name = "SalePrice"
data, target = ames_housing.drop(columns=target_name), ames_housing[target_name]
target = (target > 200_000).astype(int)

We inspect the first rows of the dataframe

In [ ]:
data

We can cherry-pick some features and only retain this subset of data

In [ ]:
numeric_features = ['LotArea', 'FullBath', 'HalfBath']
categorical_features = ['Neighborhood', 'HouseStyle']
data = data[numeric_features + categorical_features]

### Then we create the pipeline

The first step is to define the preprocessing steps

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(),
)])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

The next step is to apply the transformations using `ColumnTransformer`

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
])

Then we define the model and join the steps in order

In [ ]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression()),
])

Let's visualize it!

In [ ]:
from sklearn import set_config

set_config(display='diagram')
model

### Finally we score the model

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")